In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_probabilities_1.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)  # 5
num_samples = len(prob_df) // 3  # Assuming each model has predictions for all samples

# --- Data Preparation ---
print("=== Data Preparation ===")
print(f"Total samples: {num_samples}")
print(f"Number of classes: {num_classes}")
print(f"CSV shape: {prob_df.shape}")

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

print(f"True labels aligned: First 5 samples = {true_classes[:5]}")

# --- Model Probabilities ---
print("\n=== Model Probabilities ===")
model_probs = {}
for model_name in ['DenseNet169', 'InceptionV3', 'Xception']:
    model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
    probs = model_df[class_labels].values
    model_probs[model_name] = probs
    print(f"{model_name}:")
    print(f"  Shape: {probs.shape}")

# --- Fuzzy Ensembling ---
print("\n=== Fuzzy Ensembling Results ===")
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)
def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))
def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# Step 1: Compute membership scores for each model
model_scores = {model: compute_final_score(model_probs[model]) for model in model_probs}

# Step 2: Sum membership scores across models
ensemble_class_scores = np.zeros((num_samples, num_classes))
for model_name in model_scores:
    ensemble_class_scores += model_scores[model_name]

# Step 3: Predict classes based on summed scores
ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")

# --- Performance Evaluation ---
print("\n=== Performance Evaluation ===")
print("Classification Report:")
print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))

# --- Save Results ---
ensemble_df = pd.DataFrame({
    'True_Label': true_classes,
    'Predicted_Label': ensemble_predicted_classes
})
ensemble_df.to_csv('ensemble_predictions.csv', index=False)
print("\n=== Files Saved ===")
print("Predictions saved to: /content/ensemble_predictions.csv")


=== Data Preparation ===
Total samples: 810
Number of classes: 5
CSV shape: (2430, 8)
True labels aligned: First 5 samples = [0 1 4 1 1]

=== Model Probabilities ===
DenseNet169:
  Shape: (810, 5)
InceptionV3:
  Shape: (810, 5)
Xception:
  Shape: (810, 5)

=== Fuzzy Ensembling Results ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
Classification Report:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.91      0.93      0.92       163
            im_Koilocytotic       0.87      0.85      0.86       165
             im_Metaplastic       0.89      0.89      0.89       159
               im_Parabasal       0.92      0.91      0.92       157
im_Superficial-Intermediate       0.98      0.98      0.98       166

                   accuracy                           0.91       810
                  macro avg       0.91      0.91      0.91       810
               weighted avg       0.91      0.91      0.91    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_probabilities_1.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)  # 5
num_samples = len(prob_df) // 3  # Assuming each model has predictions for all samples

# --- Data Preparation ---
print("=== Data Preparation ===")
print(f"Total samples: {num_samples}")
print(f"Number of classes: {num_classes}")
print(f"CSV shape: {prob_df.shape}")

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

print(f"True labels aligned: First 5 samples = {true_classes[:5]}")

# --- Model Probabilities ---
print("\n=== Model Probabilities ===")
model_probs = {}
for model_name in ['DenseNet169', 'InceptionV3', 'Xception']:
    model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
    probs = model_df[class_labels].values
    model_probs[model_name] = probs
    print(f"{model_name}:")
    print(f"  Shape: {probs.shape}")

# --- Fuzzy Ensembling ---
print("\n=== Fuzzy Ensembling Results ===")
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)
def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)
def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))
def compute_final_score(probs):
    return expo_equ(probs) * norm_equ(probs)

# Step 1: Compute membership scores for each model
model_scores = {model: compute_final_score(model_probs[model]) for model in model_probs}

# Step 2: Sum membership scores across models
ensemble_class_scores = np.zeros((num_samples, num_classes))
for model_name in model_scores:
    ensemble_class_scores += model_scores[model_name]

# Step 3: Predict classes based on summed scores
ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")

# --- Performance Evaluation ---
print("\n=== Performance Evaluation ===")
print("Classification Report:")
print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))

# --- Save Results ---
ensemble_df = pd.DataFrame({
    'True_Label': true_classes,
    'Predicted_Label': ensemble_predicted_classes
})
ensemble_df.to_csv('ensemble_predictions.csv', index=False)
print("\n=== Files Saved ===")
print("Predictions saved to: /content/ensemble_predictions.csv")


=== Data Preparation ===
Total samples: 810
Number of classes: 5
CSV shape: (2430, 8)
True labels aligned: First 5 samples = [0 1 4 1 1]

=== Model Probabilities ===
DenseNet169:
  Shape: (810, 5)
InceptionV3:
  Shape: (810, 5)
Xception:
  Shape: (810, 5)

=== Fuzzy Ensembling Results ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
Classification Report:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.91      0.93      0.92       163
            im_Koilocytotic       0.86      0.85      0.86       165
             im_Metaplastic       0.89      0.89      0.89       159
               im_Parabasal       0.92      0.91      0.91       157
im_Superficial-Intermediate       0.98      0.98      0.98       166

                   accuracy                           0.91       810
                  macro avg       0.91      0.91      0.91       810
               weighted avg       0.91      0.91      0.91    

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_probabilities_1.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)  # 5
num_samples = len(prob_df) // 3  # Assuming each model has predictions for all samples

# --- Data Preparation ---
print("=== Data Preparation ===")
print(f"Total samples: {num_samples}")
print(f"Number of classes: {num_classes}")
print(f"CSV shape: {prob_df.shape}")

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

print(f"True labels aligned: First 5 samples = {true_classes[:5]}")

# --- Model Probabilities ---
print("\n=== Model Probabilities ===")
model_probs = {}
for model_name in ['DenseNet169', 'InceptionV3', 'Xception']:
    model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
    probs = model_df[class_labels].values
    model_probs[model_name] = probs
    print(f"{model_name}:")
    print(f"  Shape: {probs.shape}")

# --- Fuzzy Ensembling ---
print("\n=== Fuzzy Ensembling Results ===")
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)
def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))
def compute_final_score(probs):
    return tanh_equ(probs) * norm_equ(probs)

# Step 1: Compute membership scores for each model
model_scores = {model: compute_final_score(model_probs[model]) for model in model_probs}

# Step 2: Sum membership scores across models
ensemble_class_scores = np.zeros((num_samples, num_classes))
for model_name in model_scores:
    ensemble_class_scores += model_scores[model_name]

# Step 3: Predict classes based on summed scores
ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")

# --- Performance Evaluation ---
print("\n=== Performance Evaluation ===")
print("Classification Report:")
print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))

# --- Save Results ---
ensemble_df = pd.DataFrame({
    'True_Label': true_classes,
    'Predicted_Label': ensemble_predicted_classes
})
ensemble_df.to_csv('ensemble_predictions.csv', index=False)
print("\n=== Files Saved ===")
print("Predictions saved to: /content/ensemble_predictions.csv")


=== Data Preparation ===
Total samples: 810
Number of classes: 5
CSV shape: (2430, 8)
True labels aligned: First 5 samples = [0 1 4 1 1]

=== Model Probabilities ===
DenseNet169:
  Shape: (810, 5)
InceptionV3:
  Shape: (810, 5)
Xception:
  Shape: (810, 5)

=== Fuzzy Ensembling Results ===
First 5 predicted classes: [4 4 3 4 3]

=== Performance Evaluation ===
Classification Report:
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.00      0.00      0.00     163.0
            im_Koilocytotic       0.00      0.00      0.00     165.0
             im_Metaplastic       0.00      0.00      0.00     159.0
               im_Parabasal       0.00      0.00      0.00     157.0
im_Superficial-Intermediate       0.00      0.00      0.00     166.0

                   accuracy                           0.00     810.0
                  macro avg       0.00      0.00      0.00     810.0
               weighted avg       0.00      0.00      0.00    

In [ ]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Load combined probabilities from CSV
combined_csv_path = 'all_model_probabilities.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 8  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'InceptionResNetV2', 'Enhanced_VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 7
    for model_combo in itertools.combinations(model_list, r):
        print(f"\n=== Evaluating Combination: {model_combo} ===")
        
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        print(f"First 5 predicted classes: {ensemble_predicted_classes[:5]}")
        
        # Performance evaluation
        print("\n=== Performance Evaluation ===")
        print(classification_report(true_classes, ensemble_predicted_classes, target_names=class_labels))
        



=== Evaluating Combination: ('DenseNet169', 'InceptionV3') ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
                             precision    recall  f1-score   support

            im_Dyskeratotic       0.89      0.94      0.92       163
            im_Koilocytotic       0.84      0.83      0.84       165
             im_Metaplastic       0.90      0.88      0.89       159
               im_Parabasal       0.94      0.92      0.93       157
im_Superficial-Intermediate       0.98      0.98      0.98       166

                   accuracy                           0.91       810
                  macro avg       0.91      0.91      0.91       810
               weighted avg       0.91      0.91      0.91       810

Predictions saved to: ensemble_predictions_DenseNet169_InceptionV3.csv

=== Evaluating Combination: ('DenseNet169', 'Xception') ===
First 5 predicted classes: [0 0 4 1 1]

=== Performance Evaluation ===
                             precision

## FOLD1

In [6]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load Fold 1 probabilities from CSV
combined_csv_path = 'fold1.csv'  # Changed to Fold 1 CSV
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 7  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'InceptionResNetV2', 
              'VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# List to store combination results
combo_results = []

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 7
    for model_combo in itertools.combinations(model_list, r):
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        
        # Calculate accuracy
        acc = accuracy_score(true_classes, ensemble_predicted_classes)
        
        # Store combination result
        combo_name = ', '.join(model_combo)
        combo_results.append({
            'Combination': combo_name,
            'Fold': 'Fold_1',
            'Accuracy': acc
        })
        
        print(f"Combination: {combo_name}, Accuracy: {acc:.4f}")

# Create DataFrame from results
results_df = pd.DataFrame(combo_results)

# Get top 5 combinations by accuracy
top_5_combinations = results_df.nlargest(5, 'Accuracy')

# Save top 5 to CSV
output_csv_path = 'top_5_model_combinations_fold_1.csv'
top_5_combinations.to_csv(output_csv_path, index=False)

print("\n=== Top 5 Combinations ===")
print(top_5_combinations)
print(f"\nResults saved to {output_csv_path}")

Combination: DenseNet169, InceptionV3, Accuracy: 0.9119
Combination: DenseNet169, Xception, Accuracy: 0.9193
Combination: DenseNet169, InceptionResNetV2, Accuracy: 0.9459
Combination: DenseNet169, VGG19, Accuracy: 0.9533
Combination: DenseNet169, MobileNetV2, Accuracy: 0.9089
Combination: DenseNet169, ResNet50V2, Accuracy: 0.9237
Combination: InceptionV3, Xception, Accuracy: 0.8852
Combination: InceptionV3, InceptionResNetV2, Accuracy: 0.9333
Combination: InceptionV3, VGG19, Accuracy: 0.9370
Combination: InceptionV3, MobileNetV2, Accuracy: 0.8859
Combination: InceptionV3, ResNet50V2, Accuracy: 0.9119
Combination: Xception, InceptionResNetV2, Accuracy: 0.9259
Combination: Xception, VGG19, Accuracy: 0.9311
Combination: Xception, MobileNetV2, Accuracy: 0.8822
Combination: Xception, ResNet50V2, Accuracy: 0.9148
Combination: InceptionResNetV2, VGG19, Accuracy: 0.9281
Combination: InceptionResNetV2, MobileNetV2, Accuracy: 0.9252
Combination: InceptionResNetV2, ResNet50V2, Accuracy: 0.9511
Co

In [7]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load Fold 1 probabilities from CSV
combined_csv_path = 'fold2.csv'  # Changed to Fold 1 CSV
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 7  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'InceptionResNetV2', 
              'VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# List to store combination results
combo_results = []

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 7
    for model_combo in itertools.combinations(model_list, r):
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        
        # Calculate accuracy
        acc = accuracy_score(true_classes, ensemble_predicted_classes)
        
        # Store combination result
        combo_name = ', '.join(model_combo)
        combo_results.append({
            'Combination': combo_name,
            'Fold': 'Fold_2',
            'Accuracy': acc
        })
        
        print(f"Combination: {combo_name}, Accuracy: {acc:.4f}")

# Create DataFrame from results
results_df = pd.DataFrame(combo_results)

# Get top 5 combinations by accuracy
top_5_combinations = results_df.nlargest(5, 'Accuracy')

# Save top 5 to CSV
output_csv_path = 'top_5_model_combinations_fold_2.csv'
top_5_combinations.to_csv(output_csv_path, index=False)

print("\n=== Top 5 Combinations ===")
print(top_5_combinations)
print(f"\nResults saved to {output_csv_path}")

Combination: DenseNet169, InceptionV3, Accuracy: 0.8830
Combination: DenseNet169, Xception, Accuracy: 0.8874
Combination: DenseNet169, InceptionResNetV2, Accuracy: 0.9526
Combination: DenseNet169, VGG19, Accuracy: 0.9437
Combination: DenseNet169, MobileNetV2, Accuracy: 0.8785
Combination: DenseNet169, ResNet50V2, Accuracy: 0.9089
Combination: InceptionV3, Xception, Accuracy: 0.8667
Combination: InceptionV3, InceptionResNetV2, Accuracy: 0.9333
Combination: InceptionV3, VGG19, Accuracy: 0.9341
Combination: InceptionV3, MobileNetV2, Accuracy: 0.8630
Combination: InceptionV3, ResNet50V2, Accuracy: 0.8941
Combination: Xception, InceptionResNetV2, Accuracy: 0.9378
Combination: Xception, VGG19, Accuracy: 0.9311
Combination: Xception, MobileNetV2, Accuracy: 0.8570
Combination: Xception, ResNet50V2, Accuracy: 0.8874
Combination: InceptionResNetV2, VGG19, Accuracy: 0.9393
Combination: InceptionResNetV2, MobileNetV2, Accuracy: 0.9252
Combination: InceptionResNetV2, ResNet50V2, Accuracy: 0.9541
Co

In [8]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# Load Fold 1 probabilities from CSV
combined_csv_path = 'fold3.csv'  # Changed to Fold 1 CSV
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 7  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'InceptionResNetV2', 
              'VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# List to store combination results
combo_results = []

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 7
    for model_combo in itertools.combinations(model_list, r):
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        
        # Calculate accuracy
        acc = accuracy_score(true_classes, ensemble_predicted_classes)
        
        # Store combination result
        combo_name = ', '.join(model_combo)
        combo_results.append({
            'Combination': combo_name,
            'Fold': 'Fold_3',
            'Accuracy': acc
        })
        
        print(f"Combination: {combo_name}, Accuracy: {acc:.4f}")

# Create DataFrame from results
results_df = pd.DataFrame(combo_results)

# Get top 5 combinations by accuracy
top_5_combinations = results_df.nlargest(5, 'Accuracy')

# Save top 5 to CSV
output_csv_path = 'top_5_model_combinations_fold_3.csv'
top_5_combinations.to_csv(output_csv_path, index=False)

print("\n=== Top 5 Combinations ===")
print(top_5_combinations)
print(f"\nResults saved to {output_csv_path}")

Combination: DenseNet169, InceptionV3, Accuracy: 0.9051
Combination: DenseNet169, Xception, Accuracy: 0.9021
Combination: DenseNet169, InceptionResNetV2, Accuracy: 0.9533
Combination: DenseNet169, VGG19, Accuracy: 0.9451
Combination: DenseNet169, MobileNetV2, Accuracy: 0.8866
Combination: DenseNet169, ResNet50V2, Accuracy: 0.9014
Combination: InceptionV3, Xception, Accuracy: 0.8814
Combination: InceptionV3, InceptionResNetV2, Accuracy: 0.9288
Combination: InceptionV3, VGG19, Accuracy: 0.9362
Combination: InceptionV3, MobileNetV2, Accuracy: 0.8762
Combination: InceptionV3, ResNet50V2, Accuracy: 0.8947
Combination: Xception, InceptionResNetV2, Accuracy: 0.9362
Combination: Xception, VGG19, Accuracy: 0.9259
Combination: Xception, MobileNetV2, Accuracy: 0.8769
Combination: Xception, ResNet50V2, Accuracy: 0.8933
Combination: InceptionResNetV2, VGG19, Accuracy: 0.9451
Combination: InceptionResNetV2, MobileNetV2, Accuracy: 0.9274
Combination: InceptionResNetV2, ResNet50V2, Accuracy: 0.9444
Co

In [18]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Load Fold 1 probabilities from CSV
combined_csv_path = 'fold1.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 7  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'InceptionResNetV2', 
              'VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# List to store combination results
combo_results = []

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 7
    for model_combo in itertools.combinations(model_list, r):
        combo_name = ', '.join(model_combo)
        print(f"Evaluating Combination: {combo_name}")
        
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        
        # Generate classification report
        report = classification_report(true_classes, ensemble_predicted_classes, 
                                     target_names=class_labels, output_dict=False)
        
        # Calculate accuracy
        acc = accuracy_score(true_classes, ensemble_predicted_classes)
        
        # Store combination result
        combo_results.append({
            'Combination': combo_name,
            'Fold': 'Fold_1',
            'Accuracy': acc,
            'Report': report
        })
        
        print(f"Combination: {combo_name}, Accuracy: {acc:.4f}")

# Create DataFrame from results
results_df = pd.DataFrame(combo_results)

# Get top 10 combinations by accuracy
top_10_combinations = results_df.nlargest(10, 'Accuracy')

# Initialize docx document
doc = Document()
doc.add_heading('Top 10 Model Combinations - Classification Reports (Fold 1)', 0)

# Style settings for neat structure
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(10)

# Add top 10 reports to docx
for idx, row in top_10_combinations.iterrows():
    # Add combination heading
    heading = doc.add_heading(level=1)
    heading_run = heading.add_run(f"Rank {idx + 1}: {row['Combination']} (Accuracy: {row['Accuracy']:.4f})")
    heading_run.font.name = 'Arial'
    heading_run.font.size = Pt(12)
    
    # Add classification report label
    doc.add_paragraph('Classification Report:', style='Normal')
    
    # Add report with fixed-width font for alignment
    report_para = doc.add_paragraph()
    report_run = report_para.add_run(row['Report'])
    report_run.font.name = 'Courier New'  # Monospaced for table-like alignment
    report_run.font.size = Pt(9)
    report_para.paragraph_format.space_after = Pt(6)
    
    # Add separator
    doc.add_paragraph('-' * 50, style='Normal')

# Save docx
output_docx_path = 'top_10_model_combinations_fold_1.docx'
doc.save(output_docx_path)

# Get top 5 combinations for CSV
top_5_combinations = results_df.nlargest(5, 'Accuracy')[['Combination', 'Fold', 'Accuracy']]

# Save top 5 to CSV
output_csv_path = 'top_5_model_combinations_fold_1.csv'
top_5_combinations.to_csv(output_csv_path, index=False)

print(f"\n=== Top 5 Combinations ===")
print(top_5_combinations)
print(f"\nTop 10 classification reports saved to {output_docx_path}")
print(f"Top 5 combinations saved to {output_csv_path}")

Evaluating Combination: DenseNet169, InceptionV3
Combination: DenseNet169, InceptionV3, Accuracy: 0.9119
Evaluating Combination: DenseNet169, Xception
Combination: DenseNet169, Xception, Accuracy: 0.9193
Evaluating Combination: DenseNet169, InceptionResNetV2
Combination: DenseNet169, InceptionResNetV2, Accuracy: 0.9459
Evaluating Combination: DenseNet169, VGG19
Combination: DenseNet169, VGG19, Accuracy: 0.9533
Evaluating Combination: DenseNet169, MobileNetV2
Combination: DenseNet169, MobileNetV2, Accuracy: 0.9089
Evaluating Combination: DenseNet169, ResNet50V2
Combination: DenseNet169, ResNet50V2, Accuracy: 0.9237
Evaluating Combination: InceptionV3, Xception
Combination: InceptionV3, Xception, Accuracy: 0.8852
Evaluating Combination: InceptionV3, InceptionResNetV2
Combination: InceptionV3, InceptionResNetV2, Accuracy: 0.9333
Evaluating Combination: InceptionV3, VGG19
Combination: InceptionV3, VGG19, Accuracy: 0.9370
Evaluating Combination: InceptionV3, MobileNetV2
Combination: Incepti

## Fold2


In [21]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Load Fold 1 probabilities from CSV
combined_csv_path = 'fold2.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 7  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'InceptionResNetV2', 
              'VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# List to store combination results
combo_results = []

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 7
    for model_combo in itertools.combinations(model_list, r):
        combo_name = ', '.join(model_combo)
        print(f"Evaluating Combination: {combo_name}")
        
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        
        # Generate classification report
        report = classification_report(true_classes, ensemble_predicted_classes, 
                                     target_names=class_labels, output_dict=False)
        
        # Calculate accuracy
        acc = accuracy_score(true_classes, ensemble_predicted_classes)
        
        # Store combination result
        combo_results.append({
            'Combination': combo_name,
            'Fold': 'Fold_1',
            'Accuracy': acc,
            'Report': report
        })
        
        print(f"Combination: {combo_name}, Accuracy: {acc:.4f}")

# Create DataFrame from results
results_df = pd.DataFrame(combo_results)

# Get top 10 combinations by accuracy
top_10_combinations = results_df.nlargest(10, 'Accuracy')

# Initialize docx document
doc = Document()
doc.add_heading('Top 10 Model Combinations - Classification Reports (Fold 2)', 0)

# Style settings for neat structure
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(10)

# Add top 10 reports to docx
for idx, row in top_10_combinations.iterrows():
    # Add combination heading
    heading = doc.add_heading(level=1)
    heading_run = heading.add_run(f"Rank {idx + 1}: {row['Combination']} (Accuracy: {row['Accuracy']:.4f})")
    heading_run.font.name = 'Arial'
    heading_run.font.size = Pt(12)
    
    # Add classification report label
    doc.add_paragraph('Classification Report:', style='Normal')
    
    # Add report with fixed-width font for alignment
    report_para = doc.add_paragraph()
    report_run = report_para.add_run(row['Report'])
    report_run.font.name = 'Courier New'  # Monospaced for table-like alignment
    report_run.font.size = Pt(9)
    report_para.paragraph_format.space_after = Pt(6)
    
    # Add separator
    doc.add_paragraph('-' * 50, style='Normal')

# Save docx
output_docx_path = 'top_10_model_combinations_fold_2.docx'
doc.save(output_docx_path)


print(f"\nTop 10 classification reports saved to {output_docx_path}")

Evaluating Combination: DenseNet169, InceptionV3
Combination: DenseNet169, InceptionV3, Accuracy: 0.8830
Evaluating Combination: DenseNet169, Xception
Combination: DenseNet169, Xception, Accuracy: 0.8874
Evaluating Combination: DenseNet169, InceptionResNetV2
Combination: DenseNet169, InceptionResNetV2, Accuracy: 0.9526
Evaluating Combination: DenseNet169, VGG19
Combination: DenseNet169, VGG19, Accuracy: 0.9437
Evaluating Combination: DenseNet169, MobileNetV2
Combination: DenseNet169, MobileNetV2, Accuracy: 0.8785
Evaluating Combination: DenseNet169, ResNet50V2
Combination: DenseNet169, ResNet50V2, Accuracy: 0.9089
Evaluating Combination: InceptionV3, Xception
Combination: InceptionV3, Xception, Accuracy: 0.8667
Evaluating Combination: InceptionV3, InceptionResNetV2
Combination: InceptionV3, InceptionResNetV2, Accuracy: 0.9333
Evaluating Combination: InceptionV3, VGG19
Combination: InceptionV3, VGG19, Accuracy: 0.9341
Evaluating Combination: InceptionV3, MobileNetV2
Combination: Incepti

## Fold3

In [22]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Load Fold 1 probabilities from CSV
combined_csv_path = 'fold3.csv'
prob_df = pd.read_csv(combined_csv_path)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)
num_samples = len(prob_df) // 7  # Assuming each model has predictions for all samples

# Extract true labels directly from CSV
true_classes_from_csv = prob_df['True_Label'].unique()
if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
    le = LabelEncoder()
    le.fit(class_labels)
    true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
else:
    true_classes = prob_df['True_Label'].iloc[:num_samples].values

# Available models
model_list = ['DenseNet169', 'InceptionV3', 'Xception', 'InceptionResNetV2', 
              'VGG19', 'MobileNetV2', 'ResNet50V2']

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# List to store combination results
combo_results = []

# Generate all model combinations (starting from 2 models)
for r in range(2, len(model_list) + 1):  # r = 2 to 7
    for model_combo in itertools.combinations(model_list, r):
        combo_name = ', '.join(model_combo)
        print(f"Evaluating Combination: {combo_name}")
        
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores across selected models
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        
        # Generate classification report
        report = classification_report(true_classes, ensemble_predicted_classes, 
                                     target_names=class_labels, output_dict=False)
        
        # Calculate accuracy
        acc = accuracy_score(true_classes, ensemble_predicted_classes)
        
        # Store combination result
        combo_results.append({
            'Combination': combo_name,
            'Fold': 'Fold_1',
            'Accuracy': acc,
            'Report': report
        })
        
        print(f"Combination: {combo_name}, Accuracy: {acc:.4f}")

# Create DataFrame from results
results_df = pd.DataFrame(combo_results)

# Get top 10 combinations by accuracy
top_10_combinations = results_df.nlargest(10, 'Accuracy')

# Initialize docx document
doc = Document()
doc.add_heading('Top 10 Model Combinations - Classification Reports (Fold 3)', 0)

# Style settings for neat structure
style = doc.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(10)

# Add top 10 reports to docx
for idx, row in top_10_combinations.iterrows():
    # Add combination heading
    heading = doc.add_heading(level=1)
    heading_run = heading.add_run(f"Rank {idx + 1}: {row['Combination']} (Accuracy: {row['Accuracy']:.4f})")
    heading_run.font.name = 'Arial'
    heading_run.font.size = Pt(12)
    
    # Add classification report label
    doc.add_paragraph('Classification Report:', style='Normal')
    
    # Add report with fixed-width font for alignment
    report_para = doc.add_paragraph()
    report_run = report_para.add_run(row['Report'])
    report_run.font.name = 'Courier New'  # Monospaced for table-like alignment
    report_run.font.size = Pt(9)
    report_para.paragraph_format.space_after = Pt(6)
    
    # Add separator
    doc.add_paragraph('-' * 50, style='Normal')

# Save docx
output_docx_path = 'top_10_model_combinations_fold_3.docx'
doc.save(output_docx_path)


print(f"\nTop 10 classification reports saved to {output_docx_path}")

Evaluating Combination: DenseNet169, InceptionV3
Combination: DenseNet169, InceptionV3, Accuracy: 0.9051
Evaluating Combination: DenseNet169, Xception
Combination: DenseNet169, Xception, Accuracy: 0.9021
Evaluating Combination: DenseNet169, InceptionResNetV2
Combination: DenseNet169, InceptionResNetV2, Accuracy: 0.9533
Evaluating Combination: DenseNet169, VGG19
Combination: DenseNet169, VGG19, Accuracy: 0.9451
Evaluating Combination: DenseNet169, MobileNetV2
Combination: DenseNet169, MobileNetV2, Accuracy: 0.8866
Evaluating Combination: DenseNet169, ResNet50V2
Combination: DenseNet169, ResNet50V2, Accuracy: 0.9014
Evaluating Combination: InceptionV3, Xception
Combination: InceptionV3, Xception, Accuracy: 0.8814
Evaluating Combination: InceptionV3, InceptionResNetV2
Combination: InceptionV3, InceptionResNetV2, Accuracy: 0.9288
Evaluating Combination: InceptionV3, VGG19
Combination: InceptionV3, VGG19, Accuracy: 0.9362
Evaluating Combination: InceptionV3, MobileNetV2
Combination: Incepti

In [12]:
import itertools
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Fuzzy membership functions
def expo_equ(probs):
    return 1 - np.exp(-((probs - 1) ** 2) / 2)

def tanh_equ(probs):
    return 1 - np.tanh(((probs - 1) ** 2) / 2)

def norm_equ(probs):
    return 1 / (1 + np.exp(-probs))

def compute_final_score(probs):
    return expo_equ(probs) * tanh_equ(probs) * norm_equ(probs)

# Class labels
class_labels = ['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic',
                'im_Parabasal', 'im_Superficial-Intermediate']
num_classes = len(class_labels)

# Load top 5 combinations from Fold 1
top_5_csv_path = 'top_5_model_combinations_fold_1.csv'
top_5_df = pd.read_csv(top_5_csv_path)
top_5_combinations = top_5_df['Combination'].tolist()

# Process each fold (Fold 2 and Fold 3)
for fold in [2, 3]:
    print(f"\n=== Processing Fold {fold} ===")
    
    # Load fold probabilities
    combined_csv_path = f'fold{fold}.csv'
    prob_df = pd.read_csv(combined_csv_path)
    
    # Calculate number of samples
    num_samples = len(prob_df) // 8  # Assuming each model has predictions for all samples
    
    # Extract true labels
    true_classes_from_csv = prob_df['True_Label'].unique()
    if isinstance(true_classes_from_csv[0], str):  # If labels are strings, encode them
        le = LabelEncoder()
        le.fit(class_labels)
        true_classes = le.transform(prob_df['True_Label'].iloc[:num_samples].values)
    else:
        true_classes = prob_df['True_Label'].iloc[:num_samples].values
    
    # Initialize results for this fold
    combo_results = []
    
    # Evaluate top 5 combinations
    for combo_name in top_5_combinations:
        print(f"Evaluating Combination: {combo_name}")
        
        # Parse model names from combination
        model_combo = [model.strip() for model in combo_name.split(',')]
        
        # Extract model probabilities
        model_probs = {}
        for model_name in model_combo:
            model_df = prob_df[prob_df['Model'] == model_name].iloc[:num_samples]
            probs = model_df[class_labels].values
            model_probs[model_name] = probs
        
        # Compute membership scores
        model_scores = {}
        for model in model_probs:
            model_scores[model] = compute_final_score(model_probs[model])
        
        # Sum membership scores
        ensemble_class_scores = np.zeros((num_samples, num_classes))
        for model_name in model_scores:
            ensemble_class_scores += model_scores[model_name]
        
        # Predict classes
        ensemble_predicted_classes = np.argmin(ensemble_class_scores, axis=1)
        
        # Generate classification report
        report = classification_report(true_classes, ensemble_predicted_classes, 
                                     target_names=class_labels, output_dict=False)
        
        # Calculate accuracy
        acc = accuracy_score(true_classes, ensemble_predicted_classes)
        
        # Store result
        combo_results.append({
            'Combination': combo_name,
            'Fold': f'Fold_{fold}',
            'Accuracy': acc,
            'Report': report
        })
        
        print(f"Combination: {combo_name}, Accuracy: {acc:.4f}")
    
    # Create DataFrame from results
    results_df = pd.DataFrame(combo_results)
    
    # Initialize docx for this fold
    doc = Document()
    doc.add_heading(f'Top Model Combinations - Classification Reports (Fold {fold})', 0)
    
    # Style settings for neat structure
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Arial'
    font.size = Pt(10)
    
    # Add reports to docx
    for idx, row in results_df.iterrows():
        # Add combination heading
        heading = doc.add_heading(level=1)
        heading_run = heading.add_run(f"Rank {idx + 1}: {row['Combination']} (Accuracy: {row['Accuracy']:.4f})")
        heading_run.font.name = 'Arial'
        heading_run.font.size = Pt(12)
        
        # Add classification report label
        doc.add_paragraph('Classification Report:', style='Normal')
        
        # Add report with fixed-width font
        report_para = doc.add_paragraph()
        report_run = report_para.add_run(row['Report'])
        report_run.font.name = 'Courier New'  # Monospaced for alignment
        report_run.font.size = Pt(9)
        report_para.paragraph_format.space_after = Pt(6)
        
        # Add separator
        doc.add_paragraph('-' * 50, style='Normal')
    
    # Save docx
    output_docx_path = f'top_model_combinations_fold_{fold}.docx'
    doc.save(output_docx_path)
    
    # Save accuracies to CSV
    output_csv_path = f'top_5_accuracies_fold_{fold}.csv'
    results_df[['Combination', 'Fold', 'Accuracy']].to_csv(output_csv_path, index=False)
    
    print(f"\nClassification reports saved to {output_docx_path}")
    print(f"Accuracies saved to {output_csv_path}")
    print(f"Results for Fold {fold}:\n{results_df[['Combination', 'Fold', 'Accuracy']]}")


=== Processing Fold 2 ===
Evaluating Combination: DenseNet169, InceptionResNetV2, VGG19, ResNet50V2
Combination: DenseNet169, InceptionResNetV2, VGG19, ResNet50V2, Accuracy: 0.9788
Evaluating Combination: InceptionResNetV2, VGG19, ResNet50V2
Combination: InceptionResNetV2, VGG19, ResNet50V2, Accuracy: 0.9805
Evaluating Combination: InceptionResNetV2, VGG19, MobileNetV2, ResNet50V2
Combination: InceptionResNetV2, VGG19, MobileNetV2, ResNet50V2, Accuracy: 0.9788
Evaluating Combination: DenseNet169, InceptionResNetV2, VGG19, MobileNetV2
Combination: DenseNet169, InceptionResNetV2, VGG19, MobileNetV2, Accuracy: 0.9721
Evaluating Combination: DenseNet169, InceptionResNetV2, VGG19, MobileNetV2, ResNet50V2
Combination: DenseNet169, InceptionResNetV2, VGG19, MobileNetV2, ResNet50V2, Accuracy: 0.9687

Classification reports saved to top_model_combinations_fold_2.docx
Accuracies saved to top_5_accuracies_fold_2.csv
Results for Fold 2:
                                         Combination    Fold